In [42]:
# Import Libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [3]:
# Get the URL response from Link
url = 'http://www.imdb.com/search/title?release_date=2017&sort=num_votes,desc&page=1'
response = requests.get(url, headers = {"Accept-Language": "en-US, en;q=0.5"})

print (response.content[:500])

b'\n\n\n<!DOCTYPE html>\n<html\n    xmlns:og="http://ogp.me/ns#"\n    xmlns:fb="http://www.facebook.com/2008/fbml">\n    <head>\n         \n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">\n\n\n\n        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:\'java\'};</script>\n\n<script>\n    if (typeof uet == \'function\') {\n      uet("bb", "LoadTitle"'


In [4]:
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [12]:
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


In [29]:
first_movie = movie_containers[0]
# print(first_movie)

In [8]:
first_name = first_movie.h3.a.text
print(first_name)

Logan


In [10]:
first_year = first_movie.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
print(first_year)

(2017)


In [16]:
first_imdb = float(first_movie.strong.text)
print (first_imdb)

8.1


In [17]:
first_mscore = first_movie.find('span', class_ = 'metascore favorable')
first_mscore = int(first_mscore.text)
print(first_mscore)

77


In [20]:
first_votes = first_movie.find('span', attrs = {'name':'nv'})
first_votes = int(first_votes['data-value'])
print(first_votes)

576223


In [22]:
eighth_movie_mscore = movie_containers[22].find('div', class_ = 'ratings-metascore')
type(eighth_movie_mscore)

NoneType

In [25]:
# Lists to store the scraped data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# Extract data from individual movie container
for container in movie_containers:
    # If the movie has Metascore, then extract:
    if container.find('div', class_ = 'ratings-metascore') is not None:
        # The name
        name = container.h3.a.text
        names.append(name)
        # The year
        year = container.h3.find('span', class_ = 'lister-item-year').text
        years.append(year)
        # The IMDB rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        # The Metascore
        m_score = container.find('span', class_ = 'metascore').text
        metascores.append(int(m_score))
        # The number of votes
        vote = container.find('span', attrs = {'name':'nv'})['data-value']
        votes.append(int(vote))

In [28]:
test_df = pd.DataFrame({'movie': names,
'year': years,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes
})
print(test_df.info())
test_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 5 columns):
movie        43 non-null object
year         43 non-null object
imdb         43 non-null float64
metascore    43 non-null int64
votes        43 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.8+ KB
None


,movie,year,imdb,metascore,votes
0,Logan,(2017),8.1,77,576223
1,Thor: Ragnarok,(2017),7.9,74,507728
2,Guardians of the Galaxy Vol. 2,(2017),7.6,67,502562
3,Wonder Woman,(2017),7.4,76,500089
4,Dunkirk,(2017),7.9,94,479959
5,Star Wars: Episode VIII - The Last Jedi,(2017),7.1,85,474887
6,Spider-Man: Homecoming,(2017),7.4,73,451212
7,Get Out,(I) (2017),7.7,84,419264
8,Blade Runner 2049,(2017),8.0,81,390868
9,It,(I) (2017),7.4,69,387276


In [35]:
# Do iterations between webpages
pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(2000,2018)]
from time import time
from time import sleep
from random import randint

start_time = time()
requests = 0
for _ in range(5):
# A request would go here
    requests += 1
    sleep(randint(1,3))
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))

Request: 1; Frequency: 0.9949199662216656 requests/s
Request: 2; Frequency: 0.6653483867145237 requests/s
Request: 3; Frequency: 0.5986604337922985 requests/s
Request: 4; Frequency: 0.49905777546365865 requests/s
Request: 5; Frequency: 0.4537045678745843 requests/s


In [37]:
from IPython.core.display import clear_output
start_time = time()
requests = 0
for _ in range(5):
# A request would go here
    requests += 1
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
clear_output(wait = True)

Request: 1; Frequency: 0.49875937543775034 requests/s
Request: 2; Frequency: 0.49889769324119343 requests/s
Request: 3; Frequency: 0.5984358187006913 requests/s
Request: 4; Frequency: 0.5701685742842568 requests/s
Request: 5; Frequency: 0.5544959260987974 requests/s


In [38]:
from warnings import warn
warn("Warning Simulation")

/Users/jindongyang/.virtualenvs/hubble/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Warning Simulation
  


In [45]:
from requests import get

headers = {"Accept-Language": "en-US, en;q=0.5"}

# Redeclaring the lists to store data in
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# Preparing the monitoring of the loop
start_time = time()
requests = 0

pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(2000,2018)]

# For every year in the interval 2000-2017
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        response = get('http://www.imdb.com/search/title?release_date=' + year_url +
        '&sort=num_votes,desc&page=' + page, headers = headers)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 72:
            warn('Number of requests was greater than expected.')
            break

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the number of votes
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))

Request:72; Frequency: 0.07750735977748392 requests/s
